<a href="https://colab.research.google.com/github/ClaudioLucioLopes/PrescriptiveAnalysis/blob/master/5_0_Recomenda%C3%A7%C3%A3o_com_estoque_V01_RESPOSTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de Recomendação com limitação do estoque 

# Instalação e módulos necessários

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp
import numpy as np


# Organizando os dados

In [0]:
#Matriz dos escore de clientes 
todos_escores = np.array([ [13,5,10,7,11,1,12,5],
                                   [8,7,1,11,11,0,0,1],
                                   [4,6,9,8,11,6,7,11],
                                   [13,5,11,1,7,10,11,5],
                                   [11,9,5,1,6,3,1,3],
                                   [9,11,8,14,7,5,6,1],
                                   [9,2,5,4,9,0,10,1],
                                   [14,2,13,3,10,6,11,10],
                                   [6,0,14,5,11,5,1,0],
                                   [10,5,0,14,5,9,6,10],       
                                   [12,6,11,7,0,13,3,5],
                                   [12,7,4,4,13,10,12,14],
                                   [9,5,7,1,8,5,9,13],
                                   [11,9,5,6,2,1,1,2],
                                   [10,1,0,3,13,12,14,7],
                                   [3,4,4,0,5,8,1,11],
                                   [0,12,2,14,10,14,7,5],
                                   [10,12,14,4,4,1,9,11],     
                                   [2,6,1,10,11,8,4,4],
                                   [11,10,10,6,4,6,5,3]])


#Número de clientes e produtos no instante t
numero_clientes = todos_escores.shape[0]
numero_produtos = todos_escores.shape[1]

#Limites do estoque por produto
Limites_estoque = np.array([2, 1, 3, 0, 10, 4, 2, 4])
#Para simplificar, considere que temos estoque para fazer recomendações para todos os clientes



# Criando o modelo

In [0]:
# Criando o modelo de investimento
modelo_recomendacao_estoque = pywraplp.Solver('Modelo para maximizar a recomendação no estoque',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

#Criando as variáveis de decisão
recomendacao = {}
for i in range(numero_clientes):
    for j in range(numero_produtos):
        recomendacao[i,j] = modelo_recomendacao_estoque.BoolVar( 'recomendacao[%i,%i]' % (i,j))
print(recomendacao)

#Criando a nossa função objetivo
max_tot_score = modelo_recomendacao_estoque.Sum([todos_escores[i,j] * recomendacao[i,j]   for i in range(numero_clientes)  for j in range(numero_produtos)])    

#Atribuindo a maximização
objetivo = modelo_recomendacao_estoque.Maximize(max_tot_score)

#Restrição para que todo cliente tenham pelo menos uma recomendação de produto
for i in range(numero_clientes):
    modelo_recomendacao_estoque.Add(modelo_recomendacao_estoque.Sum([recomendacao[i,j] for j in range(numero_produtos)]) == 1)

#Restrição do estoque do produto para o número de recomendações
for j in range(numero_produtos):
    modelo_recomendacao_estoque.Add(modelo_recomendacao_estoque.Sum([recomendacao[i,j] for i in range(numero_clientes)]) <= Limites_estoque[j])


modelo_recomendacao_estoque.Add(modelo_recomendacao_estoque.Sum([recomendacao[i,j] for i in range(numero_clientes) for j in range(numero_produtos)]) == numero_clientes)


{(0, 0): recomendacao[0,0], (0, 1): recomendacao[0,1], (0, 2): recomendacao[0,2], (0, 3): recomendacao[0,3], (0, 4): recomendacao[0,4], (0, 5): recomendacao[0,5], (0, 6): recomendacao[0,6], (0, 7): recomendacao[0,7], (1, 0): recomendacao[1,0], (1, 1): recomendacao[1,1], (1, 2): recomendacao[1,2], (1, 3): recomendacao[1,3], (1, 4): recomendacao[1,4], (1, 5): recomendacao[1,5], (1, 6): recomendacao[1,6], (1, 7): recomendacao[1,7], (2, 0): recomendacao[2,0], (2, 1): recomendacao[2,1], (2, 2): recomendacao[2,2], (2, 3): recomendacao[2,3], (2, 4): recomendacao[2,4], (2, 5): recomendacao[2,5], (2, 6): recomendacao[2,6], (2, 7): recomendacao[2,7], (3, 0): recomendacao[3,0], (3, 1): recomendacao[3,1], (3, 2): recomendacao[3,2], (3, 3): recomendacao[3,3], (3, 4): recomendacao[3,4], (3, 5): recomendacao[3,5], (3, 6): recomendacao[3,6], (3, 7): recomendacao[3,7], (4, 0): recomendacao[4,0], (4, 1): recomendacao[4,1], (4, 2): recomendacao[4,2], (4, 3): recomendacao[4,3], (4, 4): recomendacao[4,4], 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7ffa843ec240> >

# Executando o modelo

In [0]:
#Executando o modelo
print('Número de restrições: ',modelo_recomendacao_estoque.NumConstraints())
print('Número de variáveis: ',modelo_recomendacao_estoque.NumVariables())

modelo_recomendacao_estoque.Solve()

print('Tempo: ', modelo_recomendacao_estoque.WallTime(), 'ms')
print('Iterações: ', modelo_recomendacao_estoque.iterations())

Número de restrições:  29
Número de variáveis:  160
Tempo:  44 ms
Iterações:  0


# Avaliando os resultados

In [0]:
print('Score total máximo da recomendação: ', round(modelo_recomendacao_estoque.Objective().Value(),2))

total_produtos_recomendados = np.zeros(numero_produtos)


for i in range(numero_clientes):
    for j in range(numero_produtos):
        if recomendacao[i,j].SolutionValue() > 0:
            total_produtos_recomendados[j] += 1
            print('Cliente %i: recebeu a recomendação do produto: %i' % (i+1,j+1))

print()
print('Produtos recomendados: ',total_produtos_recomendados.astype(int))
print('Total produtos recomendados:',np.sum(total_produtos_recomendados.astype(int)))
print()
print('Limites do Estoque:             ',Limites_estoque)
print('Total limite:',np.sum(Limites_estoque))

Score total máximo da recomendação:  234.0
Cliente 1: recebeu a recomendação do produto: 5
Cliente 2: recebeu a recomendação do produto: 5
Cliente 3: recebeu a recomendação do produto: 5
Cliente 4: recebeu a recomendação do produto: 6
Cliente 5: recebeu a recomendação do produto: 1
Cliente 6: recebeu a recomendação do produto: 2
Cliente 7: recebeu a recomendação do produto: 5
Cliente 8: recebeu a recomendação do produto: 7
Cliente 9: recebeu a recomendação do produto: 3
Cliente 10: recebeu a recomendação do produto: 8
Cliente 11: recebeu a recomendação do produto: 6
Cliente 12: recebeu a recomendação do produto: 8
Cliente 13: recebeu a recomendação do produto: 8
Cliente 14: recebeu a recomendação do produto: 1
Cliente 15: recebeu a recomendação do produto: 7
Cliente 16: recebeu a recomendação do produto: 8
Cliente 17: recebeu a recomendação do produto: 6
Cliente 18: recebeu a recomendação do produto: 3
Cliente 19: recebeu a recomendação do produto: 5
Cliente 20: recebeu a recomendação 